In [13]:
import json
import os
from datetime import datetime

import dotenv
from pymongo import MongoClient

dotenv.load_dotenv(override=True)

LOCAL_MONGO_URI = os.getenv("MONGO_URI")
SERVER_MONGO_URI = os.getenv("MONGO_URI_SERVER")

local_client = MongoClient(LOCAL_MONGO_URI)
server_client = MongoClient(SERVER_MONGO_URI)

# Carrega as categorias do JSON
with open("../../agents-categories.local.json", "r") as f:
    categories = json.load(f)

print(f"📋 Carregadas {len(categories)} categorias do JSON")


📋 Carregadas 23 categorias do JSON


In [14]:
def sync_categories_to_db(client, db_name="LibreChat"):
    """Sincroniza categorias do JSON para o MongoDB"""
    db = client[db_name]
    collection = db.agentcategories

    print(f"🔄 Sincronizando categorias para {db_name}...")

    # Limpa todas as categorias existentes
    result = collection.delete_many({})
    print(f"🗑️  Removidas {result.deleted_count} categorias existentes")

    # Adiciona timestamps às categorias
    categories_with_timestamps = []
    for category in categories:
        category_copy = category.copy()
        category_copy["createdAt"] = datetime.utcnow()
        category_copy["updatedAt"] = datetime.utcnow()
        categories_with_timestamps.append(category_copy)

    # Insere as novas categorias
    if categories_with_timestamps:
        result = collection.insert_many(categories_with_timestamps)
        print(f"✅ Inseridas {len(result.inserted_ids)} novas categorias")

        # Mostra as categorias inseridas
        print("\n📋 Categorias sincronizadas:")
        for cat in categories:
            status = "🟢 Ativa" if cat.get("isActive", True) else "🔴 Inativa"
            custom = "🔧 Personalizada" if cat.get("custom", False) else "📦 Padrão"
            print(f"  • {cat['value']:<20} | {cat['label']:<30} | Ordem: {cat.get('order', 0):<2} | {status} | {custom}")
    else:
        print("⚠️  Nenhuma categoria para sincronizar")

    print(f"✅ Sincronização concluída para {db_name}\n")


In [15]:
sync_categories_to_db(local_client, "LibreChat")
sync_categories_to_db(server_client, "LibreChat")

🔄 Sincronizando categorias para LibreChat...
🗑️  Removidas 23 categorias existentes
✅ Inseridas 23 novas categorias

📋 Categorias sincronizadas:
  • geral                | 🪶 Geral                        | Ordem: 0  | 🟢 Ativa | 🔧 Personalizada
  • motores-montagem     | 🔧 Montagem                     | Ordem: 1  | 🟢 Ativa | 🔧 Personalizada
  • body-shop-pintura    | 🚙 BS & Pintura                 | Ordem: 1  | 🟢 Ativa | 🔧 Personalizada
  • logistica            | 🚚 Logística                    | Ordem: 2  | 🟢 Ativa | 🔧 Personalizada
  • qualidade            | ✅ Qualidade                    | Ordem: 3  | 🟢 Ativa | 🔧 Personalizada
  • planejamento-industrial | 📈 Plan. Industrial             | Ordem: 4  | 🟢 Ativa | 🔧 Personalizada
  • manutencao           | 🛠️ Manutenção                  | Ordem: 5  | 🟢 Ativa | 🔧 Personalizada
  • planejamento-estrategico | 🧭 Plan. Estratégico            | Ordem: 6  | 🟢 Ativa | 🔧 Personalizada
  • engenharia           | 🛠️ Engenharia                  | Orde

/tmp/ipykernel_20874/3310274287.py:16: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  category_copy["createdAt"] = datetime.utcnow()
/tmp/ipykernel_20874/3310274287.py:17: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  category_copy["updatedAt"] = datetime.utcnow()
